In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
xl = pd.ExcelFile(r"Desktop\banhanh\2018\proA\2017 S2 Raw data Comments_Final Clean File.xlsx")
df = xl.parse("Comments")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in df['Comments']]

In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [4]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=5, id2word = dictionary, passes=50)
ldamodel.save('model5.gensim')

In [5]:
print(ldamodel.print_topics(num_topics=5, num_words=5))

[(0, '0.117*"lecture" + 0.060*"tutorial" + 0.032*"class" + 0.026*"time" + 0.024*"hour"'), (1, '0.021*"lab" + 0.021*"would" + 0.018*"like" + 0.018*"class" + 0.017*"could"'), (2, '0.036*"assignment" + 0.036*"student" + 0.027*"work" + 0.023*"group" + 0.021*"project"'), (3, '0.060*"unit" + 0.053*"good" + 0.035*"well" + 0.026*"content" + 0.022*"really"'), (4, '0.016*"assignment" + 0.015*"need" + 0.015*"unit" + 0.015*"student" + 0.014*"question"')]


In [6]:
dictionary = gensim.corpora.Dictionary.load(r'dictionary.gensim')
corpus = pickle.load(open(r'corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load(r'model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\ProgramData\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\ProgramData\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\ProgramData\Anaconda3\lib\site-packages\funcy\flow.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Hashable
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by de